# Setup

## Import modules

In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
import pprint
import tempfile

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

2024-09-02 22:19:38.633155: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-02 22:19:38.662003: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-02 22:19:38.662047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-02 22:19:38.663025: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-02 22:19:38.668189: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-02 22:19:38.669184: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [ ]:
from tensorflow.keras import layers

In [4]:
from typing import Dict, Text

In [5]:
from datetime import datetime

In [6]:
import re

## Import data

In [52]:
current_directory = os.getcwd()
data_directory = os.path.join(current_directory, '..', 'raw_data')
data_directory = os.path.abspath(data_directory)
data_directory

'/home/marcel/code/m-r-c-l/Movie-Recommendation-Engine/raw_data'

In [53]:
# Load each CSV file into a DataFrame
links_df = pd.read_csv(os.path.join(data_directory, 'ml-latest-small/links.csv'))
ratings_df = pd.read_csv(os.path.join(data_directory, 'ml-latest-small/ratings.csv'))
tags_df = pd.read_csv(os.path.join(data_directory, 'ml-latest-small/tags.csv'))
movies_df = pd.read_csv(os.path.join(data_directory, 'ml-latest-small/movies.csv'))

## Prepare data

### Convert timestamp data into datetime data

In [54]:
ratings_df['date'] = ratings_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x))
ratings_df.drop('timestamp', axis=1, inplace=True)

### Merge movies_df with ratings_df

In [55]:
## Merge movies_df with ratings_df
merged_df = ratings_df.merge(movies_df[['movieId', 'title', 'genres']], left_on='movieId',right_on='movieId', how='left')

### Convert user and title to string

In [56]:
merged_df['userId'] = merged_df['userId'].astype(str)
merged_df['title'] = merged_df['title'].astype(str)

### Clean title data

In [57]:
def clean_title(title):
    # Removing year in parentheses
    title = re.sub(r'\(\d{4}\)', '', title)
    # Removing parentheses
    title = title.replace('()', '')
    # Striping extra spaces
    return title.strip()

In [59]:
merged_df['movie_title'] = merged_df['title'].apply(clean_title)
merged_df.drop('title', axis=1, inplace=True)

### Rename columns

In [60]:
merged_df = merged_df.rename(columns={'userId': 'user_id', 'movieId': 'movie_id'})

### Convert df into tf

In [62]:
data_dict = {
    'user_id': tf.convert_to_tensor(merged_df['user_id'].values),
    'rating': tf.convert_to_tensor(merged_df['rating'].values),
    'movie_title': tf.convert_to_tensor(merged_df['movie_title'].values)
}

dataset = tf.data.Dataset.from_tensor_slices(data_dict)

In [63]:
# Select the basic features.
ratings = dataset.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})

movies = dataset.map(lambda x: x["movie_title"])

#### Validate that the transformation worked

In [64]:
for rating in ratings.take(5):
    print(rating)

{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Toy Story'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'1'>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Grumpier Old Men'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'1'>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Heat'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'1'>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Seven (a.k.a. Se7en)'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'1'>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Usual Suspects, The'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'1'>}


In [80]:
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'movie_title': b'Toy Story', 'user_id': b'1'}


In [65]:
for movie in movies.take(5):
    print(movie)

tf.Tensor(b'Toy Story', shape=(), dtype=string)
tf.Tensor(b'Grumpier Old Men', shape=(), dtype=string)
tf.Tensor(b'Heat', shape=(), dtype=string)
tf.Tensor(b'Seven (a.k.a. Se7en)', shape=(), dtype=string)
tf.Tensor(b'Usual Suspects, The', shape=(), dtype=string)


In [81]:
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

b'Toy Story'


# The models

## Model I (based on TensorFlow tutorial)

### Train, test, split

In [67]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

### Create batches

In [68]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000).map(lambda x: x["user_id"])

array([b"'71", b"'Hellboy': The Seeds of Creation", b"'Round Midnight",
       b"'Salem's Lot", b"'Til There Was You",
       b"'Tis the Season for Love", b"'burbs, The", b"'night Mother",
       b'(500) Days of Summer', b'*batteries not included'], dtype=object)

### Get unique user_ids and movie_titles 

This is important because we need to be able to map the raw values of our categorical features to embedding vectors in our models. To do that, we need a vocabulary that maps a raw feature value to an integer in a contiguous range: this allows us to look up the corresponding embeddings in our embedding tables.

In [ ]:
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

### Model setup

Choosing the architecture of our model is a key part of modelling.

Because we are building a two-tower retrieval model, we can build each tower separately and then combine them in the final model.

#### The query tower

The first step is to decide on the dimensionality of the query and candidate representations. Higher values will correspond to models that may be more accurate, but will also be slower to fit and more prone to overfitting.


In [69]:
embedding_dimension = 32

The second is to define the model itself. Here, we're going to use Keras preprocessing layers to first convert user ids to integers, and then convert those to user embeddings via an Embedding layer. Note that we use the list of unique user ids we computed earlier as a vocabulary:

A simple model like this corresponds exactly to a classic matrix factorization approach. While defining a subclass of tf.keras.Model for this simple model might be overkill, we can easily extend it to an arbitrarily complex model using standard Keras components, as long as we return an embedding_dimension-wide output at the end.

In [70]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

#### The candidate tower

We can do the same with the candidate tower.

In [71]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

#### Metrics

In our training data we have positive (user, movie) pairs. To figure out how good our model is, we need to compare the affinity score that the model calculates for this pair to the scores of all the other possible candidates: if the score for the positive pair is higher than for all other candidates, our model is highly accurate.

To do this, we can use the tfrs.metrics.FactorizedTopK metric. The metric has one required argument: the dataset of candidates that are used as implicit negatives for evaluation.

In our case, that's the movies dataset, converted into embeddings via our movie model:

In [72]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

#### Loss

The next component is the loss used to train our model. TFRS has several loss layers and tasks to make this easy.

In this instance, we'll make use of the Retrieval task object: a convenience wrapper that bundles together the loss function and metric computation.

The task itself is a Keras layer that takes the query and candidate embeddings as arguments, and returns the computed loss: we'll use that to implement the model's training loop.

In [73]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

#### The full model

We can now put it all together into a model. TFRS exposes a base model class (tfrs.models.Model) which streamlines building models: all we need to do is to set up the components in the __init__ method, and implement the compute_loss method, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.

The tfrs.Model base class is a simply convenience class: it allows us to compute both training and test losses using the same method.

In [74]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

### Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [75]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [76]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

Then train the model:

In [77]:
model.fit(cached_train, epochs=3)

Epoch 1/3


2024-09-02 22:44:05.186418: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 268435456 exceeds 10% of free system memory.
2024-09-02 22:44:05.279542: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 268435456 exceeds 10% of free system memory.


 1/10 [==>...........................] - ETA: 1:37 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 2.4414e-04 - factorized_top_k/top_100_categorical_accuracy: 3.6621e-04 - loss: 73817.9844 - regularization_loss: 0.0000e+00 - total_loss: 73817.9844

2024-09-02 22:44:13.665572: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 268435456 exceeds 10% of free system memory.
2024-09-02 22:44:13.773968: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 268435456 exceeds 10% of free system memory.


 2/10 [=====>........................] - ETA: 1:10 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 1.2207e-04 - factorized_top_k/top_100_categorical_accuracy: 1.8311e-04 - loss: 73818.4336 - regularization_loss: 0.0000e+00 - total_loss: 73818.4336

2024-09-02 22:44:22.521051: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 268435456 exceeds 10% of free system memory.


10/10 [==============================] - 87s 8s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 3.7500e-05 - factorized_top_k/top_100_categorical_accuracy: 6.2500e-05 - loss: 69912.8395 - regularization_loss: 0.0000e+00 - total_loss: 69912.8395
Epoch 2/3
10/10 [==============================] - 99s 10s/step - factorized_top_k/top_1_categorical_accuracy: 6.6250e-04 - factorized_top_k/top_5_categorical_accuracy: 9.1250e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0010 - factorized_top_k/top_50_categorical_accuracy: 0.0016 - factorized_top_k/top_100_categorical_accuracy: 0.0028 - loss: 67372.8700 - regularization_loss: 0.0000e+00 - total_loss: 67372.8700
Epoch 3/3
10/10 [==============================] - 100s 10s/step - factorized_top_k/top_1_categorical_accuracy: 5.8750e-04 - factorized_top_k/top_5_cat

If you want to monitor the training process with TensorBoard, you can add a TensorBoard callback to fit() function and then start TensorBoard using %tensorboard --logdir logs/fit. Please refer to TensorBoard documentation for more details.

As the model trains, the loss is falling and a set of top-k retrieval metrics is updated. These tell us whether the true positive is in the top-k retrieved items from the entire candidate set. For example, a top-5 categorical accuracy metric of 0.2 would tell us that, on average, the true positive is in the top 5 retrieved items 20% of the time.

Note that, in this example, we evaluate the metrics during training as well as evaluation. Because this can be quite slow with large candidate sets, it may be prudent to turn metric calculation off in training, and only run it in evaluation.

Finally, we can evaluate our model on the test set:

In [78]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 25s 5s/step - factorized_top_k/top_1_categorical_accuracy: 7.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 7.5000e-04 - factorized_top_k/top_10_categorical_accuracy: 7.5000e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0016 - factorized_top_k/top_100_categorical_accuracy: 0.0029 - loss: 31083.9222 - regularization_loss: 0.0000e+00 - total_loss: 31083.9222


{'factorized_top_k/top_1_categorical_accuracy': 0.000750000006519258,
 'factorized_top_k/top_5_categorical_accuracy': 0.000750000006519258,
 'factorized_top_k/top_10_categorical_accuracy': 0.000750000006519258,
 'factorized_top_k/top_50_categorical_accuracy': 0.0015999999595806003,
 'factorized_top_k/top_100_categorical_accuracy': 0.002850000048056245,
 'loss': 28238.7421875,
 'regularization_loss': 0,
 'total_loss': 28238.7421875}

Test set performance is much worse than training performance. This is due to two factors:

1. Our model is likely to perform better on the data that it has seen, simply because it can memorize it. This overfitting phenomenon is especially strong when models have many parameters. It can be mediated by model regularization and use of user and movie features that help the model generalize better to unseen data.
2. The model is re-recommending some of users' already watched movies. These known-positive watches can crowd out test movies out of top K recommendations.

The second phenomenon can be tackled by excluding previously seen movies from test recommendations. This approach is relatively common in the recommender systems literature, but we don't follow it in these tutorials. If not recommending past watches is important, we should expect appropriately specified models to learn this behaviour automatically from past user history and contextual information. Additionally, it is often appropriate to recommend the same item multiple times (say, an evergreen TV series or a regularly purchased item).

### Making predictions

Now that we have a model, we would like to be able to make predictions. We can use the tfrs.layers.factorized_top_k.BruteForce layer to do this.

In [ ]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

### Deploying the model

Here's an interesting section in the tutorial that talk about how to make predictions faster and more efficient: https://www.tensorflow.org/recommenders/examples/basic_retrieval#model_serving

## Model II (based on Kaggle example)

In [7]:
# Select the basic features (including rating now)
ratings = dataset.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "rating": float(x["rating"])
})

movies = dataset.map(lambda x: x["movie_title"])

NameError: name 'dataset' is not defined

In [84]:
print('Total Data: {}'.format(len(ratings)))

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = ratings.take(35_000)
test = ratings.skip(35_000).take(8_188)

Total Data: 100836


In [86]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print('Unique Movies: {}'.format(len(unique_movie_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))

Unique Movies: 9446
Unique users: 610


In [87]:
class MovieModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 64

    # User and movie models.
    self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.movie_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    movie_embeddings = self.movie_model(features["movie_title"])
    
    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("rating")

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [88]:
model = MovieModel(rating_weight=1.0, retrieval_weight=1.0)

In [89]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()

model.fit(cached_train, epochs=3)

Epoch 1/3
35/35 [==============================] - 51s 1s/step - root_mean_squared_error: 1.4769 - factorized_top_k/top_1_categorical_accuracy: 1.1429e-04 - factorized_top_k/top_5_categorical_accuracy: 1.1429e-04 - factorized_top_k/top_10_categorical_accuracy: 1.1429e-04 - factorized_top_k/top_50_categorical_accuracy: 1.4286e-04 - factorized_top_k/top_100_categorical_accuracy: 3.4286e-04 - loss: 6811.1626 - regularization_loss: 0.0000e+00 - total_loss: 6811.1626
Epoch 2/3
35/35 [==============================] - 54s 2s/step - root_mean_squared_error: 1.0137 - factorized_top_k/top_1_categorical_accuracy: 4.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 4.0000e-04 - factorized_top_k/top_10_categorical_accuracy: 4.0000e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0010 - factorized_top_k/top_100_categorical_accuracy: 0.0025 - loss: 6417.8068 - regularization_loss: 0.0000e+00 - total_loss: 6417.8068
Epoch 3/3
35/35 [==============================] - 56s 2s/step - root_mean

In [90]:
metrics = model.evaluate(cached_test, return_dict=True)

print(f"\nRetrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}")

9/9 [==============================] - 12s 1s/step - root_mean_squared_error: 0.9467 - factorized_top_k/top_1_categorical_accuracy: 2.4426e-04 - factorized_top_k/top_5_categorical_accuracy: 2.4426e-04 - factorized_top_k/top_10_categorical_accuracy: 2.4426e-04 - factorized_top_k/top_50_categorical_accuracy: 4.8852e-04 - factorized_top_k/top_100_categorical_accuracy: 8.5491e-04 - loss: 5730.2383 - regularization_loss: 0.0000e+00 - total_loss: 5730.2383

Retrieval top-100 accuracy: 0.001
Ranking RMSE: 0.947


In [ ]:
#### model.save_weights('tfrs.h5')

In [ ]:
def predict_movie(user, top_n=3):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    # recommends movies out of the entire movies dataset.
    index.index_from_dataset(
      tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
    )

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]))
    
    print('Top {} recommendations for user {}:\n'.format(top_n, user))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))

def predict_rating(user, movie):
    trained_movie_embeddings, trained_user_embeddings, predicted_rating = model({
          "userId": np.array([str(user)]),
          "original_title": np.array([movie])
      })
    print("Predicted rating for {}: {}".format(movie, predicted_rating.numpy()[0][0]))